In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv('/content/Mobile_Price_Classification-220531-204702.csv')

In [ ]:
df.shape

(2000, 21)

In [ ]:
df.head(5)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,0
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,0
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [ ]:
df.isnull().sum()

,0
battery_power,0
blue,0
clock_speed,0
dual_sim,0
fc,0
four_g,0
int_memory,0
m_dep,0
mobile_wt,0
n_cores,0


In [ ]:
#check the duplicate value
df.duplicated().sum()

0

In [ ]:
#check the duplicate number of rows
duplicate_rows = df[df.duplicated()].sum()
print(duplicate_rows)


battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64


#

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Separate features and target
X = df.drop('price_range', axis=1)  # Features
y = df['price_range']  # Target (0 for low, 1 for high)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1500, 20)
(500, 20)
(1500,)
(500,)


#Train the model

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

# create the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=(20,)),
    tf.keras.layers.Dense(4, activation='relu'),
])

#compli the model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.mae,
              metrics=['mae'])

#train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'x_train' is not defined

In [ ]:
#
y_pred=model.predict(x_test)
y_pred[:5]

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [ ]:
y_test.head(5)

,price_range
758,0
1097,1
642,0
1819,0
1082,1


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5030 - mae: 0.5030 
Test Accuracy: 50.80%


In [ ]:
# Save the model weights
model.save_weights('mobile_price_prediction_weights.weights.h5')


In [ ]:
import numpy as np
# Load model architecture and weights
model.load_weights('mobile_price_prediction_weights.weights.h5')

# Predict price range for new mobile specifications
new_mobile = np.array([[3, 32, 3000, 5.5, 12, 8,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])   # Example: RAM, internal memory, battery, etc.
new_mobile_scaled = scaler.transform(new_mobile)
price_range_prediction = model.predict(new_mobile_scaled)

# Interpret prediction
predicted_class = np.argmax(price_range_prediction)
if predicted_class == 0:  # Change these conditions based on your model's output
    predicted_range = 'low'
elif predicted_class == 3:  # Assuming 3 is the index for the 'high' class
    predicted_range = 'high'
else:
    predicted_range = 'medium'
print(f"Predicted Price Range: {predicted_range}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted Price Range: low


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import numpy as np
# Load model architecture and weights
model.load_weights('mobile_price_prediction_weights.weights.h5')

# Predict price range for new mobile specifications
new_mobile = np.array([[3, 32, 3000, 5.5, 12, 8,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])   # Example: RAM, internal memory, battery, etc.
new_mobile_scaled = scaler.transform(new_mobile)
price_range_prediction = model.predict(new_mobile_scaled)

# Interpret prediction
# Assuming your model outputs probabilities for 4 classes
predicted_class = np.argmax(price_range_prediction)  # Get the index of the class with the highest probability

# Map predicted class index to price range label
if predicted_class == 0:  # Change these conditions based on your model's output
    predicted_range = 'low'
elif predicted_class == 3:  # Assuming 3 is the index for the 'high' class
    predicted_range = 'high'
else:
    predicted_range = 'medium' # For example, if your classes are low, medium, medium-high, high. You need to adjust according to your labels.

print(f"Predicted Price Range: {predicted_range}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted Price Range: low


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


#second quection Task 2 – Sentiment Analysis using NLP

In [ ]:
import pandas as pd
df2=pd.read_csv('/content/Tweets.csv')


In [ ]:
df2.shape

(14640, 15)

In [ ]:
df2.isnull().sum()

,0
tweet_id,0
airline_sentiment,0
airline_sentiment_confidence,0
negativereason,5462
negativereason_confidence,4118
airline,0
airline_sentiment_gold,14600
name,0
negativereason_gold,14608
retweet_count,0


In [ ]:
df2.isnull().sum()

,0
tweet_id,0
airline_sentiment,0
airline_sentiment_confidence,0
negativereason,5462
negativereason_confidence,4118
airline,0
airline_sentiment_gold,14600
name,0
negativereason_gold,14608
retweet_count,0


In [ ]:
df2.duplicated().sum()

36

In [ ]:
#getting "airline_sentiment", "text" column value prom the data set
df2 = df2[["airline_sentiment", "text"]]

In [ ]:
df2.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


 #Preprocess Text

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download necessary NLTK resources if not already present
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()


def clean_text(text):

#data set's data convert to the Lowercase letter
  text =text.lower()
  text =nltk.word_tokenize(text)
  y=[]
  for i in text:
    if i.isalnum():
      y.append(i)

  text=y[:]
  y.clear()
#Removing stop words
  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      y.append(i)

  text=y[:]
  y.clear()

#stemming the word
  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)

# Apply the clean_text function to create the 'text_clean' column
df2['txt_clean'] = df2['text'].apply(clean_text) #This line was not being called

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
df2.head(5)

NameError: name 'df2' is not defined

In [ ]:
import pandas as pd

# Filter for negative sentiments
negative_statements = df2[df2['airline_sentiment'] == 'negative']

# Group by a relevant column (e.g., 'airline') and count negative statements
grouped_negative = negative_statements.groupby('airline_sentiment').count()

# Display the results
print(grouped_negative)

                   text  txt_clean
airline_sentiment                 
negative           9178       9178


In [ ]:
# Filter for neutral sentiments
neutral_tweets = df2[df2['airline_sentiment'] == 'neutral']

# Get the number of unique tweets
num_unique_neutral_tweets = neutral_tweets['text'].nunique()

# Display the result
print(f"Number of unique neutral tweets: {num_unique_neutral_tweets}")

Number of unique neutral tweets: 3067


# text featuring/vectorizing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_vec=TfidfVectorizer(max_features=3000)
x=tf_vec.fit_transform(df2['txt_clean']).toarray()
y=df2['airline_sentiment'].values

In [ ]:
x.shape


(14640, 3000)

# Train model

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier



model = RandomForestClassifier()

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(accuracy_score(y_test,y_pred))

0.7486338797814208
